<a href="https://colab.research.google.com/github/neotracer1/ATENEA_UNAL_403_2025/blob/main/Separaci%C3%B3n_Humedales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip -q install geopandas rasterio shapely unidecode

import os
import re
from unidecode import unidecode
import geopandas as gpd
import rasterio
from rasterio.mask import mask as rio_mask
from rasterio.features import rasterize
from shapely.geometry import mapping

# --- RUTAS (ya las tienes) ---
ruta_raster = "/content/drive/MyDrive/UNAL_MIA/ATENEA/SIG/RASTER_GEE/S2_BOGOTA_2024.tif"
shp_path    = "/content/drive/MyDrive/UNAL_MIA/ATENEA/SIG/SHAPE/ROI_HUMEDALES.shp"

# Carpetas de salida
out_dir_clip = "/content/drive/MyDrive/UNAL_MIA/ATENEA/SIG/RASTER_GEE/recortes_por_roi"
out_dir_mask = "/content/drive/MyDrive/UNAL_MIA/ATENEA/SIG/RASTER_GEE/mascaras_binarias_por_roi"
os.makedirs(out_dir_clip, exist_ok=True)
os.makedirs(out_dir_mask, exist_ok=True)

# --- Cargar ROI ---
ROI = gpd.read_file(shp_path)
assert "nombre_ap" in ROI.columns, "El shapefile no tiene la columna 'nombre_ap'."

# --- Abrir raster y alinear CRS ---
with rasterio.open(ruta_raster) as src:
    raster_crs = src.crs

# Reproyectar ROI si hace falta
if ROI.crs != raster_crs:
    ROI = ROI.to_crs(raster_crs)

# --- (Opcional pero recomendado) Disolver por 'nombre_ap' ---
# Esto une multipolígonos de un mismo nombre en una sola geometría
ROI_diss = ROI.dissolve(by="nombre_ap", as_index=False)

# Utilidad: generar nombres de archivo seguros
def slugify(s: str) -> str:
    s = unidecode(str(s)).lower()
    s = re.sub(r"[^a-z0-9]+", "_", s).strip("_")
    return s or "sin_nombre"

# --- Procesar cada polígono (por nombre_ap) ---
with rasterio.open(ruta_raster) as src:
    meta_base = src.meta.copy()
    band_count = src.count
    nodata_val = src.nodata if src.nodata is not None else 0  # define nodata si no existe

    for _, row in ROI_diss.iterrows():
        nombre = row["nombre_ap"]
        geom = row.geometry

        # Saltar geometrías vacías
        if geom is None or geom.is_empty:
            continue

        # 1) Recorte del raster original
        out_clip_name = f"{slugify(nombre)}_clip.tif"
        out_clip_path = os.path.join(out_dir_clip, out_clip_name)

        # rasterio.mask para recortar (mantiene bandas y valores)
        out_image, out_transform = rio_mask(
            src,
            [mapping(geom)],
            crop=True,
            nodata=nodata_val,   # valor de relleno fuera del polígono
            all_touched=False    # True si quieres incluir celdas tocadas por el borde
        )

        out_meta = meta_base.copy()
        out_meta.update({
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform,
            "nodata": nodata_val,
            "compress": "lzw"
        })

        with rasterio.open(out_clip_path, "w", **out_meta) as dst:
            dst.write(out_image)

        # 2) Máscara binaria del mismo tamaño/extensión del recorte
        out_mask_name = f"{slugify(nombre)}_mask.tif"
        out_mask_path = os.path.join(out_dir_mask, out_mask_name)

        # Creamos un raster 1 banda con 1 dentro del polígono y 0 fuera
        # usando la misma transform/shape del recorte
        mask_shape = (out_image.shape[1], out_image.shape[2])  # (rows, cols)
        mask_array = rasterize(
            [(geom, 1)],
            out_shape=mask_shape,
            transform=out_transform,
            fill=0,
            all_touched=False,   # True para máscara más "generosa"
            dtype="uint8"
        )

        mask_meta = {
            "driver": "GTiff",
            "height": mask_shape[0],
            "width": mask_shape[1],
            "count": 1,
            "dtype": "uint8",
            "crs": raster_crs,
            "transform": out_transform,
            "nodata": 0,
            "compress": "lzw"
        }

        with rasterio.open(out_mask_path, "w", **mask_meta) as mds:
            mds.write(mask_array, 1)

        print(f"✓ {nombre}:")
        print(f"   - Recorte guardado en: {out_clip_path}")
        print(f"   - Máscara binaria en: {out_mask_path}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 5.9 MB/s eta 0:00:00
✓ PDEM Entre Nubes-Cerro Juan Rey:
   - Recorte guardado en: /content/drive/MyDrive/UNAL_MIA/ATENEA/SIG/RASTER_GEE/recortes_por_roi/pdem_entre_nubes_cerro_juan_rey_clip.tif
   - Máscara binaria en: /content/drive/MyDrive/UNAL_MIA/ATENEA/SIG/RASTER_GEE/mascaras_binarias_por_roi/pdem_entre_nubes_cerro_juan_rey_mask.tif
✓ PDEM Entre Nubes-Cuchilla Guacamayas:
   - Recorte guardado en: /content/drive/MyDrive/UNAL_MIA/ATENEA/SIG/RASTER_GEE/recortes_por_roi/pdem_entre_nubes_cuchilla_guacamayas_clip.tif
   - Máscara binaria en: /content/drive/MyDrive/UNAL_MIA/ATENEA/SIG/RASTER_GEE/mascaras_binarias_por_roi/pdem_entre_nubes_cuchilla_guacamayas_mask.tif
✓ PDEM Entre Nubes-Cuchilla el Gavilán:
   - Recorte guardado en: /content/drive/MyDrive/UNAL_MIA/ATENEA/SIG/RASTER_GEE/recortes_por_roi/pdem_entre_nubes_cuchilla_el_gavilan_clip.tif
   - Máscara binaria en: /content/drive/MyDrive/UNAL_MIA/ATENEA/SIG/RASTER_GEE/masca